In [4]:
# Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored, cprint
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
import warnings
import sys

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

Centrado en la codificación y el escalamiento de los datos. La preparación adecuada de los datos es un paso fundamental en el desarrollo de modelos predictivos, ya que garantiza que todas las variables sean compatibles con los algoritmos utilizados. Este proceso incluye:

1.⁠ ⁠Codificación de variables categóricas. ransformación de variables categóricas en formatos numéricos, asegurando que los algoritmos puedan interpretar correctamente la información. Se seleccionará el método de codificación más adecuado según las características de las variables (nominales u ordinales) y el número de categorías.

2.⁠ ⁠Escalamiento de los datos. Normalización o estandarización de las variables numéricas para garantizar que todas operen en una misma escala, reduciendo posibles sesgos e incrementando la eficiencia del modelo.

### Importación de Funciones

In [5]:
#Funciones
import sys
sys.path.append('../src')  # Asegúrate de que ../src es la carpeta donde está Funciones_Ayuda.py
import Funciones_Ayuda as fa  # Ahora debería importarse correctamente
sys.path.remove('../src')

#Semilla 
seed = 25

### Importación de Data Set

In [11]:
df_loans_train = pd.read_csv('../data/interim/df_loans_train.csv')
df_loans_test = pd.read_csv('../data/interim/df_loans_test.csv')
df_loans_train.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,207476,340479,Revolving loans,F,N,N,0,225000.0,450000.0,22500.0,450000.0,Unaccompanied,State servant,Higher education,Single / not married,House / apartment,0.046220,-8883,-188,-4.0,-1571,NaN,1,1,0,0,0,0,Medicine staff,1.0,1,1,MONDAY,16,0,0,0,0,0,0,Military,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,291300,437460,Cash loans,M,N,N,0,85500.0,432000.0,26104.5,432000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.022625,-8183,-969,-3037.0,-822,NaN,1,1,1,1,0,0,Laborers,1.0,2,2,SUNDAY,13,0,0,0,0,0,0,Self-employed,NaN,0.168070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-326.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,300033,447584,Cash loans,F,Y,Y,1,270000.0,385749.0,17122.5,333000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,0.046220,-12042,-3827,-6046.0,-3916,9.0,1,1,0,1,1,0,Sales staff,2.0,1,1,SUNDAY,11,0,0,0,0,0,0,Self-employed,0.168106,0.745459,0.651260,NaN,NaN,0.9776,NaN,NaN,NaN,0.1379,NaN,NaN,NaN,NaN,0.0677,NaN,NaN,NaN,NaN,0.9777,NaN,NaN,NaN,0.1379,NaN,NaN,NaN,NaN,0.0706,NaN,NaN,NaN,NaN,0.9776,NaN,NaN,NaN,0.1379,NaN,NaN,NaN,NaN,0.0689,NaN,NaN,NaN,NaN,0.0678,NaN,No,2.0,0.0,2.0,0.0,-1355.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
3,202552,334805,Cash loans,F,N,Y,0,112500.0,152820.0,15016.5,135000.0

In [13]:
# Se va a eliminar la primera columna que repite el Index
df_loans_train = df_loans_train.drop('Unnamed: 0',axis=1)
df_loans_test = df_loans_test.drop('Unnamed: 0',axis=1)

In [17]:
# Diccionario de los datos:
var_description = pd.read_excel('../data/columns_description.xlsx')


In [19]:
# Llamar a la función y guardar los resultados en variables
col_bool, col_cat, col_num = fa.categorizar_columnas(df_loans_train)

### Codificación de Variables

In [22]:
cat_vars = df_loans_train.select_dtypes(include=['object']).columns

# Contar valores únicos en cada variable categórica
unique_counts = df_loans_train[cat_vars].nunique()

print(unique_counts)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64


Después de analizar el número de valores únicos de las variables categóricas y su impacto en la variable objetivo, estableceré los siguientes criterios de encoding:

Baja cardinalidad (≤ 8 categorías): Se usará One-Hot Encoding, ya que es eficiente para variables con pocas categorías y no añade complejidad innecesaria al modelo.

Media cardinalidad (9-50 categorías): No se aplicará Target Encoding debido a que las variables en este rango no muestran un impacto significativo en la relación con la variable objetivo, por lo que no justifica su uso.

En su lugar:
Para la variable OCCUPATION_TYPE (18 categorías), se aplicará Mean Encoding, ya que permite capturar de manera eficiente la relación promedio entre cada categoría y la variable objetivo.

Alta cardinalidad (> 50 categorías): Se aplicará CatBoost Encoding para la variable ORGANIZATION_TYPE (58 categorías), una técnica adecuada para manejar variables de alta cardinalidad reduciendo el riesgo de overfitting.

Este enfoque busca balancear la complejidad del modelo y la relevancia de las variables categóricas, maximizando el desempeño general del modelo.            

#### Se va a separar de nuevo en Train y Test 

In [30]:
y_train = df_loan_train['TARGET']
X_train = df_loan_train.drop('TARGET', axis=1)
y_test = df_loan_test['TARGET']
X_test = df_loan_test.drop('TARGET', axis=1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((246008, 122), (246008,), (61503, 122), (61503,))

#### CODIFICACIÓN DE ONE HOT ENCODING 

In [31]:
list_columns_cat = list(df_loans_train.select_dtypes(include=["object", "category"]).columns)
exclude_vars = ['OCCUPATION_TYPE', 'ORGANIZATION_TYPE']  # Excluir estas columnas
list_columns_ohe = [col for col in list_columns_cat if col not in exclude_vars]

# Crear y aplicar One-Hot Encoder
ohe = ce.OneHotEncoder(cols=list_columns_ohe, use_cat_names=True)
ohe.fit(X_train, y_train)  

# Transformar X_train y X_test
X_train_t = ohe.transform(X_train)
X_test_t = ohe.transform(X_test)

# Verificar formas finales
print(X_train_t.shape, X_test_t.shape)

(246008, 177) (61503, 177)


#### CODIFIACIÓN MEAN ENCODING

In [37]:
# Mean Encoding para OCCUPATION_TYPE
target_column = 'OCCUPATION_TYPE'

# Calcular el promedio de 'TARGET' por cada categoría de 'OCCUPATION_TYPE'
mean_encoding_map = X_train_t.copy()
mean_encoding_map['TARGET'] = y_train  # Añadimos la variable TARGET al DataFrame
mean_encoding_map = mean_encoding_map.groupby(target_column)['TARGET'].mean().to_dict()

# Aplicar Mean Encoding a X_train y X_test
X_train_me = X_train_t.copy()
X_test_me = X_test_t.copy()

X_train_me[target_column] = X_train_t[target_column].map(mean_encoding_map)
X_test_me[target_column] = X_test_t[target_column].map(mean_encoding_map).fillna(y_train.mean())

# Verificar formas finales
print(X_train_me.shape, X_test_me.shape)

(246008, 177) (61503, 177)


In [39]:
# CatBoost Encoding

target_column = 'ORGANIZATION_TYPE'

# Crear y ajustar el codificador de CatBoost Encoding
catboost_enc = ce.CatBoostEncoder(cols=[target_column])
catboost_enc.fit(X_train_me[target_column], y_train)  

# Transformar X_train y X_test
X_train_mec = X_train_me.copy()
X_test_mec = X_test_me.copy()

X_train_mec[target_column] = catboost_enc.transform(X_train_me[target_column])
X_test_mec[target_column] = catboost_enc.transform(X_test_me[target_column])

# Verificar formas finales
print(X_train_mec.shape, X_test_mec.shape)

(246008, 177) (61503, 177)


In [40]:
X_train_mec.dtypes.to_dict()

{'Unnamed: 0': dtype('int64'),
 'SK_ID_CURR': dtype('int64'),
 'NAME_CONTRACT_TYPE_Revolving loans': dtype('int64'),
 'NAME_CONTRACT_TYPE_Cash loans': dtype('int64'),
 'CODE_GENDER_F': dtype('int64'),
 'CODE_GENDER_M': dtype('int64'),
 'CODE_GENDER_XNA': dtype('int64'),
 'FLAG_OWN_CAR_N': dtype('int64'),
 'FLAG_OWN_CAR_Y': dtype('int64'),
 'FLAG_OWN_REALTY_N': dtype('int64'),
 'FLAG_OWN_REALTY_Y': dtype('int64'),
 'CNT_CHILDREN': dtype('int64'),
 'AMT_INCOME_TOTAL': dtype('float64'),
 'AMT_CREDIT': dtype('float64'),
 'AMT_ANNUITY': dtype('float64'),
 'AMT_GOODS_PRICE': dtype('float64'),
 'NAME_TYPE_SUITE_Unaccompanied': dtype('int64'),
 'NAME_TYPE_SUITE_Family': dtype('int64'),
 'NAME_TYPE_SUITE_Other_A': dtype('int64'),
 'NAME_TYPE_SUITE_Children': dtype('int64'),
 'NAME_TYPE_SUITE_Spouse, partner': dtype('int64'),
 'NAME_TYPE_SUITE_Other_B': dtype('int64'),
 'NAME_TYPE_SUITE_Group of people': dtype('int64'),
 'NAME_TYPE_SUITE_nan': dtype('int64'),
 'NAME_INCOME_TYPE_State servant': d

#### ESCALADO DE VARIABLES

In [41]:
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_mec)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_mec), columns=X_train_mec.columns, index=X_train_mec.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_mec), columns=X_test_mec.columns, index=X_test_mec.index)

In [44]:
X_train_scaled  = X_train_scaled.drop('Unnamed: 0',axis=1)
X_test_scaled = X_test_scaled.drop('Unnamed: 0',axis=1)

In [45]:
X_train_scaled.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Children,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_nan,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Maternity leave,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_terraced house,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_nan,TOTALAREA_MODE,WALLSMATERIAL_MODE_Others,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,0.606703,3.079472,-3.079472,0.7

In [46]:
X_train_scaled.describe()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Children,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_nan,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Maternity leave,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_terraced house,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_nan,TOTALAREA_MODE,WALLSMATERIAL_MODE_Others,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,2.460080e+05,2.460080e+05,2

In [47]:
X_test_scaled.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Children,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_nan,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Maternity leave,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_terraced house,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_nan,TOTALAREA_MODE,WALLSMATERIAL_MODE_Others,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1.692551,-0.324731,0.324731,0.7

In [48]:
X_test_scaled.describe()

,SK_ID_CURR,NAME_CONTRACT_TYPE_Revolving loans,NAME_CONTRACT_TYPE_Cash loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE_Unaccompanied,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Children,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Other_B,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_nan,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Working,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Maternity leave,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Academic degree,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Widow,NAME_FAMILY_STATUS_Unknown,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_With parents,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Co-op apartment,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_nan,HOUSETYPE_MODE_terraced house,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_nan,TOTALAREA_MODE,WALLSMATERIAL_MODE_Others,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Panel,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,61503.000000,61503.000000,6

#### CONCLUSIONES FINALES DEL EDA

A continuación, se presenta un resumen de lo realizado hasta el momento:

Exploración y análisis inicial (EDA): Comenzamos con un análisis exploratorio para comprender el tamaño y las características de nuestro conjunto de datos, prestando especial atención a la variable objetivo, TARGET, y sus relaciones con las demás variables.

Análisis de tipos de variables: Analizamos las variables en términos de su tipo (categóricas, numéricas, etc.) y evaluamos cómo se relacionan con la variable objetivo. Esto nos ayudó a determinar qué técnicas de codificación eran más adecuadas para cada tipo de variable.

Análisis de correlación: Estudiamos las relaciones de correlación entre las variables para identificar posibles redundancias y relaciones lineales importantes que podrían influir en la selección de características.

División en conjuntos de entrenamiento y prueba: Realizamos una división estratificada de los datos en conjuntos de entrenamiento y prueba, asegurando que la distribución de la variable objetivo se mantuviera en ambas particiones.

Tratamiento de outliers: Detectamos y tratamos los valores atípicos en las variables numéricas, asegurándonos de que no afectaran negativamente al modelo.

Imputación de valores faltantes: Abordamos los valores nulos en el conjunto de datos mediante técnicas de imputación adecuadas, garantizando que las variables estuvieran completas para el entrenamiento del modelo.

Análisis WoE e IV: Aplicamos el análisis de Weight of Evidence (WoE) y la Información de Valor (IV) a las variables categóricas para medir su capacidad predictiva y determinar qué variables serían más útiles para el modelado.

Codificación de variables categóricas: Utilizamos técnicas de codificación numérica como One-Hot Encoding, Mean Encoding y CatBoost Encoding para transformar las variables categóricas en valores numéricos adecuados para los modelos.

Escalado de variables: Procedimos con el escalado de las variables numéricas para asegurar que todas las características tuvieran un rango similar y fueran adecuadamente procesadas por los modelos de Machine Learning.

Con estos pasos completados, nuestros datos están listos para continuar con las siguientes fases del proyecto: la selección de características (feature processing), la construcción y evaluación de modelos, la implementación del modelo final, y la explicación y conclusiones del trabajo realizado.